## Flights and Domestic Airlines 
Providing the best recommendations for domestic travel in the U.S. In our general applications we are using 2015 data to determine delay information and predict which airlines contains the best flights. 

We are also determining if the weather that day was the reason for the flight cancellation or weather the airline was at fault.

Finally with flight. We are looking into Hotel Recommendations and determining the best hotels domestically based on reviews.


In [5]:
import pandas as pd; import numpy as np, os

In [13]:
# Get Flight Delay information and cleanse the data for missing values

# Airports CSV
airport_info = pd.read_csv("airports.csv", sep = ",")

# Airlines CSV
airlines_info = pd.read_csv("airlines.csv", sep = ",")
airlines_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
IATA_CODE    14 non-null object
AIRLINE      14 non-null object
dtypes: object(2)
memory usage: 152.0+ bytes


In [ ]:
# Flight information (1157049 Rows)
chunksize = 500
for chunk in pd.read_csv("flights.csv", sep= ",", chunksize=chunksize): # read in chunk sizes
    chunk